# 🚀 Token Processing in LLMs: Step-by-Step Demonstration
### 📌 Understand how text is transformed inside an LLM, from Token IDs to Self-Attention
This notebook will guide you through:
- ✅ Tokenization (Converting text into Token IDs)
- ✅ Embedding Layer (Mapping tokens to dense vectors)
- ✅ Positional Encoding (Adding order information)
- ✅ Self-Attention (Computing relationships between words)
- ✅ Multi-Head Attention & Feedforward Network


In [1]:
# Install required libraries
!pip install transformers numpy torch matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 21.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 25.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy-1.13.2:
      Successfully uninstalled sympy-1.13.2


In [2]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel
import math

## 🔹 Define Sample Input Text & Tokenization
We'll tokenize the sentence into Token IDs.

In [3]:
# Load a tokenizer (GPT-2 as an example)
tokenizer = AutoTokenizer.from_pretrained('gpt2')

# Define sample text
text = "The cat sat on the mat"
tokens = tokenizer.tokenize(text)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print("🔹 Tokens:", tokens)
print("🔹 Token IDs:", token_ids)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

🔹 Tokens: ['The', 'Ġcat', 'Ġsat', 'Ġon', 'Ġthe', 'Ġmat']
🔹 Token IDs: [464, 3797, 3332, 319, 262, 2603]


## 🔹 Embedding Layer: Converting Token IDs into Dense Vectors
Each token ID is mapped to a vector in a high-dimensional space.

In [5]:
# Define an embedding layer (Assume embedding size is 8 for demonstration)
embedding_layer = nn.Embedding(num_embeddings=tokenizer.vocab_size, embedding_dim=8)
embedded_tokens = embedding_layer(torch.tensor(token_ids))

print("🔹 Embedded Token Shape:", embedded_tokens.shape)
print("🔹 First Token Embedding:", embedded_tokens[0].detach().numpy())

🔹 Embedded Token Shape: torch.Size([6, 8])
🔹 First Token Embedding: [-0.574144    1.139774    0.03128648 -0.6340446  -0.907468    1.9675677
  2.0192716   0.7869233 ]


## 🔹 Positional Encoding: Adding Order Information to Tokens
Since Transformers process words in parallel, positional encoding ensures word order is retained.

In [6]:
def positional_encoding(seq_length, d_model):
    pe = np.zeros((seq_length, d_model))
    for pos in range(seq_length):
        for i in range(0, d_model, 2):
            pe[pos, i] = math.sin(pos / (10000 ** (i / d_model)))
            pe[pos, i + 1] = math.cos(pos / (10000 ** (i / d_model)))
    return torch.tensor(pe, dtype=torch.float32)

# Apply positional encoding
pos_encoding = positional_encoding(len(token_ids), 8)
print("🔹 Positional Encoding Shape:", pos_encoding.shape)
print("🔹 First Positional Encoding Vector:", pos_encoding[0].numpy())

🔹 Positional Encoding Shape: torch.Size([6, 8])
🔹 First Positional Encoding Vector: [0. 1. 0. 1. 0. 1. 0. 1.]


## 🔹 Self-Attention Mechanism: Computing Word Relationships
We compute Query, Key, and Value matrices to find word importance.

In [7]:
d_model = 8  # Embedding dimension

# Define Q, K, V transformation matrices
W_Q = nn.Linear(d_model, d_model)
W_K = nn.Linear(d_model, d_model)
W_V = nn.Linear(d_model, d_model)

# Compute Q, K, V
Q = W_Q(embedded_tokens)
K = W_K(embedded_tokens)
V = W_V(embedded_tokens)

# Compute Attention Scores (Scaled Dot Product)
scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_model)
attention_weights = torch.nn.functional.softmax(scores, dim=-1)

print("🔹 Attention Scores Shape:", scores.shape)
print("🔹 Attention Weights:", attention_weights.detach().numpy())

🔹 Attention Scores Shape: torch.Size([6, 6])
🔹 Attention Weights: [[0.11329384 0.16812378 0.16182631 0.15167604 0.1856198  0.21946025]
 [0.15508711 0.1691271  0.16654402 0.16095573 0.16701701 0.18126905]
 [0.1568301  0.1694073  0.17542073 0.12855673 0.14569059 0.22409458]
 [0.14045294 0.16277742 0.19190936 0.1230769  0.19387445 0.18790884]
 [0.23203024 0.21496472 0.18042354 0.10720392 0.11903882 0.14633878]
 [0.19308832 0.17930296 0.16438776 0.15497737 0.12292111 0.18532243]]


## 🔹 Apply Attention to Value V
The weighted sum of values determines the final representation.

In [8]:
output = torch.matmul(attention_weights, V)
print("🔹 Attention Output Shape:", output.shape)

🔹 Attention Output Shape: torch.Size([6, 8])


## 🔹 Multi-Head Attention Simulation
Instead of one attention mechanism, we use multiple heads.

In [9]:
num_heads = 2  # Simulating two attention heads
multi_head_output = torch.cat([output, output], dim=-1)
print("🔹 Multi-Head Output Shape:", multi_head_output.shape)

🔹 Multi-Head Output Shape: torch.Size([6, 16])


## 🔹 Feedforward Layer: Refining Representations
Final transformation to enrich context before moving to the next Transformer block.

In [10]:
feedforward = nn.Sequential(
    nn.Linear(d_model * 2, 32),
    nn.ReLU(),
    nn.Linear(32, d_model * 2)
)

final_representation = feedforward(multi_head_output)
print("🔹 Final Representation Shape:", final_representation.shape)

🔹 Final Representation Shape: torch.Size([6, 16])


## 🎯 Summary: How Text is Processed in an LLM
- ✅ Tokenization → Converts text into token IDs
- ✅ Embedding Layer → Maps tokens into dense numerical vectors
- ✅ Positional Encoding → Adds order information using sine & cosine waves
- ✅ Self-Attention → Determines which words are important in context
- ✅ Multi-Head Attention → Enhances word relationships from different perspectives
- ✅ Feedforward Network → Further refines contextual embeddings

🚀 **This is the foundation of how Transformers process text!**